In [385]:
import string
import re
import os
import pandas as pd
import csv
from thefuzz import process

In [80]:
tg_path="."
tg_name="TG263_Nomenclature_Worksheet_20170815(TG263 v20170815).csv"
df = pd.read_csv(os.path.join(tg_path,tg_name))
tg_names = df['TG263-Primary Name'].to_list()
tg_names_rev = df['TG-263-Reverse Order Name'].to_list()

In [108]:
target_df = pd.read_csv('/mnt/iDriveShare/Kayla/unique_targets.csv')
target_names = target_df['Name'].to_list()
target_inst = target_df['Instances'].to_list()
target_len = target_df['Length'].to_list()

# with open('/mnt/iDriveShare/Kayla/unique_targets.csv', 'r') as f:
#     list_targets = [row[0] for row in csv.reader(f)] 

In [138]:
name = 'PTV_5000'
# list_names = ['CTV_6000','CTV_7000','GTV','CTV_5600','PTV_7000-05','PTV!_6000-05',
#               'PTV_All','PTV','CTV','PTV_ALL','PTV70+PTV60','PTV60','PTV56','CTV60','CTV70','PTV!_Eval_6000','PTV2n',
#               'CTVn_6000','CTVn','opt_CTV56_crop03','PTV^Physician1','GTV_2000^ICG','PTV1','GTVp1','GTVp2','PTVn','PTVsb']
list_names = ['CT 60','GTV','CTV_5600','PTV_7000-05','PTV_All','PTV70+PTV60','PTV60','PTV!_Eval_6000','PTV2n',
              'CTVn_6000','CTVn','opt_CTV56_crop03','PTV^Physician1','GTV_2000^ICG','PTV1','GTVp1','GTVp2','PTVn','PTVsb',
             'GTV_Liver^ICG']

# note, can easily check spaces and legnth

# to do: add exmples from the TG 263 paper

In [139]:
df = pd.read_csv(os.path.join(tg_path,tg_name))
tg_names = df['TG263-Primary Name'].to_list()

In [384]:
# note, this does not work for rule 10 of the guidelines 
# TO DO: split by ^ immediately (after length and space check) and disregard everything afterwards
# thought process for suffixes: go through and remove correct parts of name in the correct ordre. 
#                               if anything is leftover without a ^, then it is wrong

# TO DO: 
# E-PTV_Ev05_xxxx	E-PTV_Ev05_xxxx
# E-PTV_xxxx	E-PTV_xxxx

def check_target_compliance(target_name):
    debug = False
    reason = ''
    #first check if there are spaces, then it's automatically not tg 263 compliant
    if ' ' in target_name:
        reason = "spaces"
        return False, reason

    if len(target_name) > 16:
        reason = "> 16 characters"
        return False, reason
    
    if target_name[0] == "z":
        reason = "Ignore after z"
        return True, reason 
    
    if "^" in target_name:
        print("^^^")
        print("before:", target_name)
        target_name = target_name[:target_name.index("^")]
        print("after slice", target_name)
        
    
    target_prefix = target_name.split("_")[0]
    target_suffix = target_name.replace(target_prefix,'')
    
    if debug:
        print("****")

        print("prefix", target_prefix)
#     print("suffix", target_suffix)

    is_correct = True
    reasoning = ''
  

    # CHECKING PREFIX
    list_allowed_prefixes = ['PTV!','GTV','CTV','ITV','IGTV','ICTV','PTV'] #rule 1
    list_allowed_classifiers = ['n', 'p', 'sb', 'par', 'v', 'vas',''] # rule 2, including none
    #note: ordering of the above mattes, PTV! sb before PTV, and '' should be last -- this is for when removing it from prefix
    
    if not target_prefix.startswith(tuple(list_allowed_prefixes)): # rule 1
        
        
        reason = "fails rule 1"
        return False, reason
    
    for p in list_allowed_prefixes: 
        if target_prefix.startswith(p):
            target_prefix = target_prefix.replace(p,'')
            break
   
    if debug:
        print("After Rule 1:",target_prefix)

    
    
    
    if target_prefix != '':
#         if target_prefix[0] == "^": # rule 9
#             if debug:
#                 print("its true, all after ^")
#             return True, reason
        
        # need to check rule 8 again, since could show up in prefix if no _
        
        if target_prefix[0].isalpha():
            compliant = False
            for c in list_allowed_classifiers:
                if target_prefix.startswith(c):
                    compliant = True
                    target_prefix = target_prefix.replace(c,'')
                    break
            if not compliant:
                reason = "Fails rule 2"
                return False, reason
        if debug:
            print("After rule 2:",target_prefix)
            
        '''
        prefix_no_digits = target_prefix.rstrip(string.digits)
        if debug:
            print("prefix no digits:",prefix_no_digits)
        if not prefix_no_digits in list_allowed_classifiers: #rule 2
            reason = "Fails rule 2"
            return False, reason
        
        
        for c in list_allowed_classifiers:
            if target_prefix.startswith(c):
                target_prefix = target_prefix.replace(c,'')
                break
        if debug:
            print("After rule 2:",target_prefix)
        '''
        
        if target_prefix != '':
            if target_prefix[0].isdigit():
                #ok this is if there is a digit
                target_prefix = target_prefix[1:]
            
            if len(target_prefix) != 0 and not bool(re.match( r'^-\d{2}$',target_prefix)):
                reason = "Fails rule 3?"
                return False, reason
        
        '''
        if target_prefix != '' and (len(target_prefix) > 1 or not target_prefix.isdigit()):
            # rule #3 --> could be tricky, saying len sb max 1 becuase i doubt the bunbers would go up to 10
            #but... there could be a random number there that isn't meant to represnet spatially distinct targets.
            reason = "fails rule 3"
            return False, reason
        '''
            
    # TO DO -- check if ends with -xx for prefix eg CTVp2-05
    
    # CHECKING SUFFIX
    
    # TO DO: SKIPPING RULES 4 AND 5 FOR NOW
    # for rule 5, check thorugh TG again, however will need to do starts with i believe to accoutn for extra dose etc put at end
    # then need to remove it and analyse rest of structure as before
    if target_suffix != '':
        # rule #5
        split_suffix = target_suffix[1:].split("_")[0]
        print("target", target_suffix)
        print("split",split_suffix)
        if target_suffix[0] == "_" and split_suffix in tg_names:
            target_suffix = target_suffix[1:].replace(split_suffix,'')

        # rule #6

   
        relative_dose_suffixes = ["_High", "_Mid","_Low"]
        if debug:
            print("suffix", target_suffix)

        numerical_dose = False
        if target_suffix.startswith(tuple(relative_dose_suffixes)):
            #ok it is relavice dose
            # TO DO: add 01 02 etc
            for r in relative_dose_suffixes:
                if target_suffix.startswith(r):
                    target_suffix = target_suffix.replace(r,'')
                    break

            placeholder = 1

        elif bool(re.match(r'^_\d{4}', target_suffix)): #cGy
            target_suffix = target_suffix[5:]
            placeholder = 2
            numerical_dose = True
    #     elif : TO DO : Gy

        if debug:
            print("After rule 6: ", target_suffix)
        if numerical_dose: #rule 7, this is optional, only if the # fx are indicated it should be with an x
            fraction_pattern = r'^x\d{1,2}'
            if bool(re.match(fraction_pattern, target_suffix)):
                fractions = True
                target_suffix = re.sub(fraction_pattern, '', target_suffix)

            if debug:
                print("After rule 7: ", target_suffix)

        #rule 8
        if bool(re.match(r'^-\d{2}', target_suffix)):
            target_suffix = target_suffix[3:]

            if debug:
                print("after rule 8:", target_suffix)

        # rule 9
    if target_suffix == '' or target_suffix[0] == "^":
#             print("ALL GOODDDDDD")
        return True, reason
    else:
#             print("FAILED~!", target_suffix)
        reason = "Suffix leftover: " + target_suffix
        return False, reason
    

            

            

In [147]:
for name in list_names:
#     print(name)
    passes, reason = check_target_compliance(name)
    print(name, passes, reason)

CT 60 False spaces
GTV True 
target _5600
split 5600
CTV_5600 True 
target _7000-05
split 7000-05
PTV_7000-05 True 
target _All
split All
PTV_All False Suffix leftover: _All
PTV70+PTV60 False Fails rule 2
PTV60 False fails rule 3
target _Eval_6000
split Eval
PTV!_Eval_6000 True 
PTV2n False Fails rule 2
target _6000
split 6000
CTVn_6000 True 
CTVn True 
opt_CTV56_crop03 False fails rule 1
^^^
before: PTV^Physician1
after slice PTV
PTV^Physician1 True 
^^^
before: GTV_2000^ICG
after slice GTV_2000
target _2000
split 2000
GTV_2000^ICG True 
PTV1 True 
GTVp1 True 
GTVp2 True 
PTVn True 
PTVsb True 
^^^
before: GTV_Liver^ICG
after slice GTV_Liver
target _Liver
split Liver
GTV_Liver^ICG True 


In [142]:
#TO DO: check for eg if Cavity-Oral-PTV is ok
pass_list = []
reason_list = []
for name in target_names:
#     print(name)
    passes, reason = check_target_compliance(name)
    pass_list.append(passes)
    reason_list.append(reason)
    
    print(name, passes, reason)

target _6000
split 6000
CTV_6000 True 
target _7000
split 7000
CTV_7000 True 
GTV True 
target _5600
split 5600
CTV_5600 True 
target _7000-05
split 7000-05
PTV_7000-05 True 
target _6000-05
split 6000-05
PTV!_6000-05 True 
target _5600-05
split 5600-05
PTV!_5600-05 True 
zPTV7000+PTV6000 False fails rule 1
Cavity_Oral-PTV False fails rule 1
target _All
split All
PTV_All False Suffix leftover: _All
PTV True 
CTV True 
target _ALL
split ALL
PTV_ALL False Suffix leftover: _ALL
PTV70+PTV60 False Fails rule 2
PTV60 False fails rule 3
PTV56 False fails rule 3
CTV60 False fails rule 3
CTV70 False fails rule 3
GTV70 False fails rule 3
CTV56 False fails rule 3
PTV70 False fails rule 3
CTV60 FC False spaces
PTV60 FC False spaces
PTV60+PTV70 False Fails rule 2
GTVn True 
CTVp True 
target _56
split 56
PTV_56 False Suffix leftover: _56
GTVp True 
target _70
split 70
CTV_70 False Suffix leftover: _70
target _56
split 56
CTV_56 False Suffix leftover: _56
target _70
split 70
PTV_70 False Suffix left

In [112]:

with open("unique_list_targets.csv","w") as f:
    writer = csv.writer(f)
    writer.writerow(["In-House Name","Instances","Length","Matches TG-263","Reason","Structure Type","Rules"])
    writer.writerows(zip(target_names, target_inst, target_len,pass_list,reason_list))


In [12]:
prefix = name.split("_")[0]

In [13]:
suffix = name.replace(prefix,'')

In [15]:
print("prefix",prefix)
print("suffix",suffix)

prefix PTV
suffix _5000


In [67]:
target_suffix = "x20x50"

fraction_pattern = r'^x\d{1,2}'
if bool(re.match(fraction_pattern, target_suffix)):
    fractions = True
    target_suffix = re.sub(fraction_pattern, '', target_suffix)


In [68]:
target_suffix

'x50'

In [114]:
ids = ['z_Air', 'LN_Neck_VI', 'LN_Neck_V_R', 'LN_Neck_V_L', 'LN_Neck_IB_R', 'LN_Neck_IB_L', 'LN_Neck_IA', 'LN_Neck_2347AB_R', 'LN_Neck_2347AB_L', 'PTV_7000-05', 'PTV!_6000-05', 'PTV!_5600-05', 'SpinalCord~_PRV5', 'SpinalCord~', 'Spc_Retrophar_R', 'Spc_Retrophar_L', 'Skin', 'PTV_7000', 'PTV!_6000', 'PTV!_5600', 'Pituitary', 'Pharynx', 'Parotid_R', 'Parotid_L', 'OpticNrv_R', 'OpticNrv_L', 'OpticChiasm', 'Musc_Constrict_S', 'Musc_Constrict_M', 'Musc_Constrict_I', 'Lung~_R', 'Lung~_L', 'Lobe_Temporal_R', 'Lobe_Temporal_L', 'LN_Neck_R', 'LN_Neck_L', 'Lips', 'Lens_R', 'Lens_L', 'Larynx', 'GTV', 'Glnd_Thyroid', 'Glnd_Submand_R', 'Glnd_Submand_L', 'Eye_R', 'Eye_L', 'Esophagus~^2cm', 'Esophagus_S', 'CTV_7000', 'CTV_6000', 'CTV_5600', 'Cochlea_R', 'Cochlea_L', 'Cavity_Oral', 'Carina', 'Brainstem_PRV05', 'Brainstem', 'Brain', 'BrachialPlex_R', 'BrachialPlex_L', 'Bones^RadCalc', 'Bone_Mandible', 'BODY', 'Cavity_Oral-PTV', 'Constrict_SM-CTV', 'Constrict_I-CTV', 'z_Parotid_NOS_L', 'z_Parotid_NOS_R', 'zPTV7000+PTV6000', 'z_PTV_All', 'z_Larynx_NOS', 'z_Avoid_5600', 'z_Avoid_6000', 'z_Ring_5600', 'z_Ring_6000', 'z_Ring_7000', 'z_Avoid_Middle', 'z_Avoid_Post', 'z_Shoulders', 'z_Avoid_Post3000', 'z_Submand_NOS_L', 'zDensityOverride', 'z_Remove1', 'z_Remove2', 'z_Remove3', 'z_Submand_NOS_R']

In [115]:
for name in ids:
#     print(name)
    passes, reason = check_target_compliance(name)
    print(name, passes, reason)
    

z_Air False fails rule 1
LN_Neck_VI False fails rule 1
LN_Neck_V_R False fails rule 1
LN_Neck_V_L False fails rule 1
LN_Neck_IB_R False fails rule 1
LN_Neck_IB_L False fails rule 1
LN_Neck_IA False fails rule 1
LN_Neck_2347AB_R False fails rule 1
LN_Neck_2347AB_L False fails rule 1
PTV_7000-05 True 
PTV!_6000-05 True 
PTV!_5600-05 True 
SpinalCord~_PRV5 False fails rule 1
SpinalCord~ False fails rule 1
Spc_Retrophar_R False fails rule 1
Spc_Retrophar_L False fails rule 1
Skin False fails rule 1
PTV_7000 True 
PTV!_6000 True 
PTV!_5600 True 
Pituitary False fails rule 1
Pharynx False fails rule 1
Parotid_R False fails rule 1
Parotid_L False fails rule 1
OpticNrv_R False fails rule 1
OpticNrv_L False fails rule 1
OpticChiasm False fails rule 1
Musc_Constrict_S False fails rule 1
Musc_Constrict_M False fails rule 1
Musc_Constrict_I False fails rule 1
Lung~_R False fails rule 1
Lung~_L False fails rule 1
Lobe_Temporal_R False fails rule 1
Lobe_Temporal_L False fails rule 1
LN_Neck_R False 

In [199]:
def check_TG_name(name, tg_names):
    original_length = len(name)
    reason = ''

    if ' ' in name:
        reason = "spaces"
        return False, reason

    if len(name) > 16:
        reason = "> 16 characters"
        return False, reason

    if "^" in name: # Rule 12, ignore custom notes after ^
        name = name[:name.index("^")]

    if name != '':
        
        #rule #14 (asssuming doesnt matter after z?)
        if name[0] =='z' or name[0] == "_":
            return True, "ignored"

        # check if names starts with a tg compliant name (NOTE TO DO THIS IS INEFFICIENT)
    # 		if not name.startswith(tuple(tg_names)): # rule 1
    # 			reason = "fails rule 1"
    # 			return False, reason
        tg_start = False
        tg_names.sort(key=len, reverse=True)
        for n in tg_names: 
            if 'PRV' in n:
                n = n.replace('_PRVxx','').replace("_PRVx",'').replace('_PRV','')
            if name.startswith(n):
    # 				if len(name) > len(n) and name[len(n)].isalpha():
    # 					continue
                tg_start = True
                name = name.replace(n,'')
                break
            elif n.endswith("s") and (name.startswith(n[:-1]+"~") or (name.startswith(n[:-1]+"_PRV") and (name.endswith("_L") or name.endswith("_R")))):
                tg_start = True
                name = name.replace(n[:-1],'')
                break

        if not tg_start:
            return False, "Does not start with a TG compliant structure name"

        #Rule 11
        if name.startswith("~"):
            # need to check about for eg STRUCT~_L_PRV or STRUCT~_PRV_L
            name = name[1:]
            if name[0:2] =="_L" or name[0:2]=="_R":
                name = name[2:]
        ''' version if not cutting outtg name from prefix
        if "~" in name:
            split_name = name.split("~")
            if split_name[0] in tg_names:
                if split_name[1].startswith("_L"):
                    name = name.replace
             and (len(split_name[1])==0 or split_name[1]=="_L" or split_name[1]=="_R":
        '''
        # rule 10
        if name.startswith("_PRV"):
            name = name.replace('_PRV','')

            # check for xx digits after PRV
            if bool(re.match(r'^\d{2}(?!\d)', name)):# if exactly 2 digits follow
                name = name[2:]
            elif bool(re.match(r'^\d{1}(?!\d)', name)) and original_length != 16:
                return False, "PRV should have 2 digits for mm since doesn't exceed 16 characters"
            elif bool(re.match(r'^\d{1}(?!\d)', name)):
                name = name[1:]

            # check if _L or _R follow PRV
            if len(name)>1 and (name[0:2] == "_L" or name[0:2] == "_R"):
                name = name[2:]




    if name == '' or name[0] == "^": # can prob remove this now
    #             print("ALL GOODDDDDD")
        return True, reason
    else:
    #             print("FAILED~!", target_suffix)
        reason = "Suffix leftover: " + name
        return False, reason


In [200]:
check_TG_name('Lung~_L', tg_names)


(True, '')

In [278]:
check_TG_name('Brainstem_PRV03', tg_names)


(True, '')

In [279]:
check_TG_name('Brainstem_PRV05', tg_names)


(True, '')

In [203]:
for name in ids:
    print(name)
    print(check_TG_name(name,tg_names))

z_Air
(True, 'ignored')
LN_Neck_VI
(False, 'Suffix leftover: _Neck_VI')
LN_Neck_V_R
(True, '')
LN_Neck_V_L
(True, '')
LN_Neck_IB_R
(True, '')
LN_Neck_IB_L
(True, '')
LN_Neck_IA
(False, 'Suffix leftover: _Neck_IA')
LN_Neck_2347AB_R
(False, 'Suffix leftover: _Neck_2347AB_R')
LN_Neck_2347AB_L
(False, 'Suffix leftover: _Neck_2347AB_L')
PTV_7000-05
(False, 'Suffix leftover: _7000-05')
PTV!_6000-05
(False, 'Suffix leftover: !_6000-05')
PTV!_5600-05
(False, 'Suffix leftover: !_5600-05')
SpinalCord~_PRV5
(True, '')
SpinalCord~
(True, '')
Spc_Retrophar_R
(True, '')
Spc_Retrophar_L
(True, '')
Skin
(True, '')
PTV_7000
(False, 'Suffix leftover: _7000')
PTV!_6000
(False, 'Suffix leftover: !_6000')
PTV!_5600
(False, 'Suffix leftover: !_5600')
Pituitary
(True, '')
Pharynx
(True, '')
Parotid_R
(True, '')
Parotid_L
(True, '')
OpticNrv_R
(True, '')
OpticNrv_L
(True, '')
OpticChiasm
(True, '')
Musc_Constrict_S
(True, '')
Musc_Constrict_M
(True, '')
Musc_Constrict_I
(True, '')
Lung~_R
(True, '')
Lung~_L
(

In [182]:
tg_names.sort(key=len, reverse=True) 

In [209]:
check_target_compliance("PTV-05")

(True, '')

In [267]:
common_mispellings = {
    'brachialplexus': 'brachialplex',
    'brstem': 'brainstem',
    'left':'L',
    'right':'R',
    'opticnerve':'opticnrv',
    'optnerv':'opticnrv',
    'optnrv':'opticnrv',
    'spine':'spinal',
    'submandibular': 'submand',
    'submandibula':'submand',
    'submndsalv':'submand',
    'surgbed':'surgicalbed',
    'templobe':'temporallobe',
    
    
    
    
}

In [454]:


def get_proposed_name(name,tg_names,use_fuzzy=True):
    
    if (name[0] == 'z' or name[0] == "_") and ' ' in name and len(name)<= 16:
        return name.replace(" ","_"), "spaces"
    

        
    name_nosymbols = re.sub(r'[0-9]','',re.sub(r'[^\w]', '', name).lower()).replace(" ","").replace("_","")
    print(name_nosymbols)
    name_nosymbols_numbers = re.sub(r'[^\w]', '', name).lower().replace(" ","").replace("_","")
    name_lower = name.lower()
    
    if any(substring in name_nosymbols for substring in common_mispellings.keys()):
        for key in common_mispellings.keys():
            if key in name_nosymbols:
                print("key",key)
                name_nosymbols = name_nosymbols.replace(key,common_mispellings[key])
                name = name.replace(key,common_mispellings[key])
                name_nosymbols_numbers = name_nosymbols_numbers.replace(key,common_mispellings[key])
                name_lower = name_lower.replace(key,common_mispellings[key])
    tg_names.sort(key=len, reverse=False)
    for tg_name in tg_names:
        # to do: inefficient, do this one time only for removeing symbols
        tg_name_nosymbols = re.sub(r'[0-9]','',re.sub(r'[^\w]', '', tg_name).lower()).replace("_","")
    #         print(tg_name_nosymbols)
    #         print()


        if name.lower() == tg_name.lower():
            return tg_name, "casing"
        elif name.lower().replace(" ", "") == tg_name.lower():
            # print()
            return tg_name, "spaces"

        elif (tg_name_nosymbols == name_nosymbols):
            if 'prv' in tg_name_nosymbols:
                two_num = True
                
                if bool(re.search(r'PRV\d{2}', name)):
                    insert_index = tg_name.find('PRV') + len('PRV') 
                    print(tg_name.find('PRV'))
                    digits_to_insert = re.search(r'PRV\d{2}', name).group(0)[-2:]
                    print(insert_index)
                    if len(tg_name) < 15:
                        new_string = tg_name[0:insert_index] + digits_to_insert + tg_name[insert_index:]
                        
                        return new_string, "error with symbols or casing" 
                    elif len(tg_name) ==15:
                        new_string = tg_name[0:insert_index] + digits_to_insert[-1] + tg_name[insert_index:]
                        return new_string, "error with PRV numbers" 
                
                elif bool(re.search(r'PRV\d{1}', name)):
                    insert_index = tg_name.find('PRV') + len('PRV') 
                    print("insert_index")
                    digits_to_insert = re.search(r'PRV\d{1}', name).group(0)[-1]
                     
                    
                    if len(tg_name) < 15:
                        new_string = tg_name[0:insert_index] + "0"+digits_to_insert + tg_name[insert_index:]
                        return new_string, "error with PRV numbers" 
                    elif len(tg_name) ==15:
                        new_string = tg_name[0:insert_index] + digits_to_insert + tg_name[insert_index:]
                        return new_string, "error with symbols or casing" 
                        
                    
            return tg_name, "symbols"

        elif name_nosymbols.replace(" ","") in tg_name_nosymbols:
            return tg_name, "Missing part of name?"
        else:
            split_name = name.lower().split("_")
            split_name.reverse()
            tg_name_rev = ''
            split_tg = tg_name.lower().split("_")
#             print(len(split_tg))
            
            if len(split_tg)>1:
                for t in reversed(split_tg):
                    tg_name_rev = tg_name_rev + t
  

                if tg_name_rev[0] == "r" or tg_name_rev[0]=='l':
                    
                    tg_name_rev = tg_name_rev[1:] +tg_name_rev[0]


            if split_name == tg_name.lower().split("_"):
                return tg_name, "Wrong order of words"
            elif tg_name_rev == name_nosymbols.lower():
                return tg_name, "Wrong ordering of words and missing _"
        
#         elif name.lower().split("_").reverse() == tg_name.lower().split("_"):
#             return tg_name, "Wrong order of words"
        '''
        elif tg_name_nosymbols in name_nosymbols.replace(" ",""):
            return tg_name, "too many letters in name??"
        '''

        '''
        elif re.sub(r'[^\w]', '', name.lower().replace(' ','')) in re.sub(r'[^\w]', '', tg_name.lower().replace(' ','')):
            return tg_name, ""
        elif re.sub(r'[^\w]', '', tg_name.lower().replace(' ','')) == re.sub(r'[^\w]', '', name.lower().replace(' ','')):
            return tg_name, ""
        '''

        # add suggestions for common mispellings, common mistakes, adding ^ in front of garbage, etc
        
        if ('avoid' in name.lower() or 'nos' in name.lower() or 'ring' in name.lower() or ('opt' in name.lower() and 'optic' not in name.lower() and 'nrv' not in name.lower() and 'nerv' not in name.lower()) ):  #what to do with opt? since optic nerve
            if len(name) == 15:
                return "z"+name.replace(" ","_"), "consider adding z in front if not used for dose eval"
            elif len(name) < 15:
                return "z_"+name.replace(" ","_"), "consider adding z in front if not used for dose eval"
            
    if use_fuzzy:
        print("using fuzzy")
        print(name_nosymbols, name_nosymbols_numbers, name_lower)
        closest_match1 = process.extractOne(name, tg_names)
        closest_match2 = process.extractOne(name_lower, tg_names)
        closest_match2 = process.extractOne(name_lower, tg_names)
        print(closest_match2)
        print(closest_match[0])
        return closest_match2[0], 'using fuzzy'
        
    return '',''

In [455]:
get_proposed_name("OPTIC_NRV_L_PRV", tg_names)

opticnrvlprv
using fuzzy
opticnrvlprv opticnrvlprv optic_nrv_l_prv
('OpticNrv_PRV', 89)
OpticNrv


('OpticNrv_PRV', 'using fuzzy')

In [451]:
get_proposed_name("BOWEL_BAG", tg_names)

bowelbag


('Bag_Bowel', 'Wrong order of words')

In [436]:
w = "CAUDA_EQUINA"

In [437]:
w.lower().split("_")

['cauda', 'equina']

In [438]:
get_proposed_name("Brainstem_PRV03", tg_names)

brainstemprv
10
13


('Brainstem_PRV03', 'error with symbols or casing')

In [439]:
re.search(r'PRV\d{2}', "BrainStem_PRV03").group(0)

'PRV03'

In [453]:
query = "OpticNrv_R_PRV03"
closest_match = process.extractOne(query, tg_names)
print(closest_match)
print(process.extract(query, tg_names,limit=3))

('OpticNrv', 90)
[('OpticNrv', 90), ('OpticNrv_R', 90), ('OpticNrv_PRV_R', 89)]
